# WIP

In [ ]:
import feyn
import pmlb
import pandas as pd

In [ ]:
#for name in pmlb.regression_dataset_names:
#    df = pmlb.fetch_data(name, local_cache_dir="/tmp/pmlb_data")
#    print(f"('{name}', {len(df)}, {len(df.columns)}),")   

In [ ]:
datasets = pd.DataFrame([
('1027_ESL', 488, 5),
('1028_SWD', 1000, 11),
('1029_LEV', 1000, 5),
('1030_ERA', 1000, 5),
('1089_USCrime', 47, 14),
('1096_FacultySalaries', 50, 5),
('1191_BNG_pbc', 1000000, 19),
('1193_BNG_lowbwt', 31104, 10),
('1196_BNG_pharynx', 1000000, 11),
('1199_BNG_echoMonths', 17496, 10),
('1201_BNG_breastTumor', 116640, 10),
('1203_BNG_pwLinear', 177147, 11),
('1595_poker', 1025010, 11),
('192_vineyard', 52, 3),
('195_auto_price', 159, 16),
('197_cpu_act', 8192, 22),
('201_pol', 15000, 49),
('207_autoPrice', 159, 16),
('210_cloud', 108, 6),
('215_2dplanes', 40768, 11),
('218_house_8L', 22784, 9),
('225_puma8NH', 8192, 9),
('227_cpu_small', 8192, 13),
('228_elusage', 55, 3),
('229_pwLinear', 200, 11),
('230_machine_cpu', 209, 7),
('294_satellite_image', 6435, 37),
('344_mv', 40768, 11),
('4544_GeographicalOriginalofMusic', 1059, 118),
('485_analcatdata_vehicle', 48, 5),
('503_wind', 6574, 15),
('505_tecator', 240, 125),
('519_vinnie', 380, 3),
('522_pm10', 500, 8),
('523_analcatdata_neavote', 100, 3),
('527_analcatdata_election2000', 67, 15),
('529_pollen', 3848, 5),
('537_houses', 20640, 9),
('542_pollution', 60, 16),
('547_no2', 500, 8),
('556_analcatdata_apnea2', 475, 4),
('557_analcatdata_apnea1', 475, 4),
('560_bodyfat', 252, 15),
('561_cpu', 209, 8),
('562_cpu_small', 8192, 13),
('564_fried', 40768, 11),
('573_cpu_act', 8192, 22),
('574_house_16H', 22784, 17),
('579_fri_c0_250_5', 250, 6),
('581_fri_c3_500_25', 500, 26),
('582_fri_c1_500_25', 500, 26),
('583_fri_c1_1000_50', 1000, 51),
('584_fri_c4_500_25', 500, 26),
('586_fri_c3_1000_25', 1000, 26),
('588_fri_c4_1000_100', 1000, 101),
('589_fri_c2_1000_25', 1000, 26),
('590_fri_c0_1000_50', 1000, 51),
('591_fri_c1_100_10', 100, 11),
('592_fri_c4_1000_25', 1000, 26),
('593_fri_c1_1000_10', 1000, 11),
('594_fri_c2_100_5', 100, 6),
('595_fri_c0_1000_10', 1000, 11),
('596_fri_c2_250_5', 250, 6),
('597_fri_c2_500_5', 500, 6),
('598_fri_c0_1000_25', 1000, 26),
('599_fri_c2_1000_5', 1000, 6),
('601_fri_c1_250_5', 250, 6),
('602_fri_c3_250_10', 250, 11),
('603_fri_c0_250_50', 250, 51),
('604_fri_c4_500_10', 500, 11),
('605_fri_c2_250_25', 250, 26),
('606_fri_c2_1000_10', 1000, 11),
('607_fri_c4_1000_50', 1000, 51),
('608_fri_c3_1000_10', 1000, 11),
('609_fri_c0_1000_5', 1000, 6),
('611_fri_c3_100_5', 100, 6),
('612_fri_c1_1000_5', 1000, 6),
('613_fri_c3_250_5', 250, 6),
('615_fri_c4_250_10', 250, 11),
('616_fri_c4_500_50', 500, 51),
('617_fri_c3_500_5', 500, 6),
('618_fri_c3_1000_50', 1000, 51),
('620_fri_c1_1000_25', 1000, 26),
('621_fri_c0_100_10', 100, 11),
('622_fri_c2_1000_50', 1000, 51),
('623_fri_c4_1000_10', 1000, 11),
('624_fri_c0_100_5', 100, 6),
('626_fri_c2_500_50', 500, 51),
('627_fri_c2_500_10', 500, 11),
('628_fri_c3_1000_5', 1000, 6),
('631_fri_c1_500_5', 500, 6),
('633_fri_c0_500_25', 500, 26),
('634_fri_c2_100_10', 100, 11),
('635_fri_c0_250_10', 250, 11),
('637_fri_c1_500_50', 500, 51),
('641_fri_c1_500_10', 500, 11),
('643_fri_c2_500_25', 500, 26),
('644_fri_c4_250_25', 250, 26),
('645_fri_c3_500_50', 500, 51),
('646_fri_c3_500_10', 500, 11),
('647_fri_c1_250_10', 250, 11),
('648_fri_c1_250_50', 250, 51),
('649_fri_c0_500_5', 500, 6),
('650_fri_c0_500_50', 500, 51),
('651_fri_c0_100_25', 100, 26),
('653_fri_c0_250_25', 250, 26),
('654_fri_c0_500_10', 500, 11),
('656_fri_c1_100_5', 100, 6),
('657_fri_c2_250_10', 250, 11),
('658_fri_c3_250_25', 250, 26),
('659_sleuth_ex1714', 47, 8),
('663_rabe_266', 120, 3),
('665_sleuth_case2002', 147, 7),
('666_rmftsa_ladata', 508, 11),
('678_visualizing_environmental', 111, 4),
('687_sleuth_ex1605', 62, 6),
('690_visualizing_galaxy', 323, 5),
('695_chatfield_4', 235, 13),
('706_sleuth_case1202', 93, 7),
('712_chscase_geyser1', 222, 3),
('banana', 5300, 3),
('titanic', 2201, 4),
],
columns=["name","n","fcount"])

In [ ]:
datasets[(datasets["n"]>1000) & (datasets["fcount"]<12) ]

# Winners: 200 samples

Linreg:
- 1193_BNG_lowbwt (0.56)
- 1199_BNG_echoMonths (0.401)
- 529_pollen (0.786)

RF:
- banana (.58, but QL did better than LR)

GB:
- 215_2dplanes (.927, but QL did better than LR) 
- 218_house_8L (.491, but QL did better than LR)
- 564_fried (.826, but QL did better than LR)

QL: 
- 1196_BNG_pharynx (.42)
- 1203_BNG_pwLinear (.505)
- 225_puma8NH (0.59)
- 344_mv (.991)
- 537_houses (.402, but LR did better after QL feature selection)
- titanic (.266, well GB was a tie within rounding error)

None (R2 < 0)
- 1201_BNG_breastTumor
- 1595_poker



In [ ]:
def get_pmlb_data(name, trainsize=200):
    df = pmlb.fetch_data(name, local_cache_dir="/tmp/pmlb_data")
    return df[0:trainsize], df[trainsize+1:]

In [ ]:
train, test = get_pmlb_data("225_puma8NH")

# Compare to the usual suspects

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


def X(df):
    return df.iloc[:,:-1]

def y(df):
    return df.iloc[:,-1]

def fit_and_r2_score(model, train, test):
    model.fit(X(train), y(train))
    return model.score(X(train), y(train)), model.score(X(test), y(test))



r2_train, r2_test = fit_and_r2_score(LinearRegression(), train, test)
print("Linear regression:\t%.3f\t%.3f" %(r2_train, r2_test))

r2_train, r2_test = fit_and_r2_score(Lasso(alpha=0.01), train, test)
print("Lasso:\t\t\t%.3f\t%.3f" %(r2_train, r2_test))

r2_train, r2_test = fit_and_r2_score(RandomForestRegressor(), train, test)
print("Random forest:\t\t%.3f\t%.3f" %(r2_train, r2_test))

r2_train, r2_test = fit_and_r2_score(GradientBoostingRegressor(n_estimators=100, min_samples_split=2, min_samples_leaf=2), train, test)
print("Gradient Boosting:\t%.3f\t%.3f" %(r2_train, r2_test))

print("---")

In [ ]:
ql = feyn.QLattice()

In [ ]:
ql.reset()
qg = ql.get_regressor(train.columns, train.columns[-1]).filter(feyn.filters.MaxEdges(8))
#qg=qg.filter(feyn.filters.Functions(["add","multiply", "log","inverse","exp","sqrt","squared"]))

for _ in range(100):
    qg.fit(train, threads=7)
    ql.update(qg.best())
    print("Train:\t", qg[0].r2_score(train), "\nTest:\t", qg[0].r2_score(test))

In [ ]:
for _ in range(10000):
    qg[0].fit(train)

In [ ]:
ref = feyn.reference.LinearRegressionModel(qg[0],train)
print("Train:\t", qg[0].r2_score(train), "\nTest:\t", qg[0].r2_score(test))
print("-- LR --")
print("Train:\t", ref.r2_score(train), "\nTest:\t", ref.r2_score(test))